#### **Others unconventionality collection**

In [0]:
import pandas as pd
import pickle
import glob
import numpy as np

import warnings
warnings.filterwarnings("ignore")
from collections import Counter

### **1. Read Tables**

In [3]:
unconventional_set = pd.read_csv("own unconventionality.csv", encoding="iso-8859-1")
unconventional_set.head(1)

,year,imdbcrewcastlink,imdbfilmtitle,imdbfilmlink,bomtitle,bomlink,descriptor1,descriptor2,descriptor3,NumOwnFilms(past five years),NumOfTwoCommonCategoriesOwnFilms(past five years)
0,2005,http://www.imdb.com/name/nm4588077/,Fantastic Four,http://www.imdb.com/title/tt0120667/,Fantastic Four (2005),http://www.boxofficemojo.com/movies/?id=fantas...,action,adventure,fantasy,3,1


### **2. Yearly feature films collection**

**Sample address for IMDb link modification:**
- Feature Film, Released between 2000-01-01 and 2005-12-31 (Sorted by Popularity Ascending)
- https://www.imdb.com/search/title/?title_type=feature&year=2000-01-01,2005-12-31&view=advanced 
- Starts from certain number: 
- https://www.imdb.com/search/title/?title_type=feature&year=2005-01-01,2005-12-31&start=1&ref_=adv_nxt


In [0]:
# collect yearly feature films descriptors 
# and stored in the dictionary for further comparison

# package use
import requests
import urllib
import urllib.request
import time
import re
import string
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
from IPython.core.display import HTML

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import copy
from collections import Counter

In [0]:
headers = ["descriptor1", "descriptor2", "descriptor3"]

# now collection all the films record within certain years range
def yearly_film_collection(year_par, start_record_par = 1, stopping = True, headers_par = headers):
  # initialization for yearly_set based on first page
  url = "https://www.imdb.com/search/title/?title_type=feature&year=" + str(year_par) + "-01-01," + str(year_par) + "-12-31&start=" + str(start_record_par) + "&ref_=adv_nxt"
  response = requests.get(url)
  soup = BeautifulSoup(response.text, "html.parser")
  filter_out = soup.find_all("span", {"class": "genre"})
  list_ = [filter_out[i].text.strip().split(", ") for i in range(len(filter_out))]
  yearly_set = pd.DataFrame(list_, columns=headers_par)

  # continue append 
  while stopping:
    start_record_par += 50
    print(start_record_par)
    url = "https://www.imdb.com/search/title/?title_type=feature&year=" + str(year_par) + "-01-01," + str(year_par) + "-12-31&start=" + str(start_record_par) + "&ref_=adv_nxt"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    filter_out = soup.find_all("span", {"class": "genre"})
    # update list_ based on current page searching result
    list_ = [filter_out[i].text.strip().split(", ") for i in range(len(filter_out))]
    if len(list_) == 0:
      stopping = False
    else:
      sub_length = max(len(l) for l in list_)
      if sub_length < 3:
        # fill in the first sub_length column
        temp_set = pd.DataFrame(list_, columns=headers_par[0:sub_length])
        # other columns set to none
        for col in headers_par[sub_length:]:
          temp_set[col] = np.nan
      else:
        temp_set = pd.DataFrame(list_, columns=headers_par)
      yearly_set = yearly_set.append(temp_set)
  # add year information
  yearly_set['year'] = year_par
  yearly_set = yearly_set.reset_index(drop=True)
  yearly_set.to_csv(str(year_par) + ".csv")
  return True

In [0]:
# an example 
yearly_film_collection(2015)

### **3. Count others unconventionality**

In [0]:
import re
# count number of movies each has
unconventional_set['NumFeatureMovies(past five years)'] = np.nan
unconventional_set['NumCommonFeatureMovies(past five years)'] = np.nan

In [75]:
total_set = pd.read_csv("2000.csv")
for i in range(2001, 2016):
  temp = pd.read_csv(str(i) + ".csv")
  total_set = total_set.append(temp)
print(len(total_set))
total_set = total_set.reset_index(drop=True)
total_set = total_set.apply(lambda x: x.astype(str).str.lower())
total_set['year'] = total_set['year'].astype(int)
total_set.head(1)

95843


,Unnamed: 0,descriptor1,descriptor2,descriptor3,year
0,0,action,adventure,comedy,2000


In [76]:
upper_bound = len(unconventional_set)
for idx in range(upper_bound):
  # set up year constraints
  year = unconventional_set['year'][idx]
  sub_total_set = total_set[(total_set['year'] >= (year-4)) & (total_set['year'] <= year)]
  unconventional_set['NumFeatureMovies(past five years)'][idx] = len(sub_total_set)
  descriptor1 = str(unconventional_set['descriptor1'][idx])
  descriptor2 = str(unconventional_set['descriptor2'][idx])
  # count not null common films
  if descriptor1 != "nan" and descriptor2 != "nan":
    temp = sub_total_set[(sub_total_set['descriptor1'] == descriptor1) & (sub_total_set['descriptor2'] == descriptor2)]
    unconventional_set['NumCommonFeatureMovies(past five years)'][idx] = len(temp)
  if idx%200 == 0:
    print(idx)

0
200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400


In [0]:
unconventional_set.to_csv("result.csv")